# 04_02_check_stars_using_DAOFinder


## 필요한 환경

이 프로젝트를 위해서는 아래의 모듈이 필요합니다.

> numpy, pandas, matplotlib, scipy, astropy, photutils, ccdproc, ysfitsutilpy, ysphotutilpy, version_information

### 모듈 버전 확인

아래 셀을 실행하면 이 노트북을 실행한 파이썬 및 관련 모듈의 버전을 확인할 수 있다.

In [5]:
import importlib, sys, subprocess
packages = "numpy, pandas, matplotlib, scipy, astropy, ccdproc, ysfitsutilpy, ysphotutilpy, version_information" # required modules
pkgs = packages.split(", ")

for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        print(f"**** module {pkg} is not installed... now start install")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
        print(f"****** module {pkg} is installed")
    else: 
        print(f"**** module {pkg} is installed")

%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

**** module numpy is installed
**** module pandas is installed
**** module matplotlib is installed
**** module scipy is installed
**** module astropy is installed
**** module ccdproc is installed
**** module ysfitsutilpy is installed
**** module ysphotutilpy is installed
**** module version_information is installed
The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information
This notebook was generated at 2024-11-30 16:29:37 (KST = GMT+0900) 
0 Python     3.12.5 64bit [GCC 11.2.0]
1 IPython    8.27.0
2 OS         Linux 6.8.0 49 generic x86_64 with glibc2.39
3 numpy      1.26.4
4 pandas     2.2.3
5 matplotlib 3.9.2
6 scipy      1.14.1
7 astropy    6.1.4
8 ccdproc    2.4.2
9 ysfitsutilpy 0.2
10 ysphotutilpy 0.1.1
11 version_information 1.0.4


### import modules

In [6]:
import os
from glob import glob
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import rcParams
from mpl_toolkits.axes_grid1 import make_axes_locatable

from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.nddata import CCDData
import astropy.units as u
from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from astropy.stats import sigma_clip, sigma_clipped_stats

from photutils import DAOStarFinder
from photutils import IRAFStarFinder
from photutils.aperture import CircularAperture as CAp
from photutils.aperture import CircularAnnulus as CAn
from photutils import detect_threshold
from photutils.centroids import centroid_com
#from photutils import aperture_photometry as apphot

import warnings

from ccdproc import CCDData, ccd_process

from astropy.time import Time
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord

import ysfitsutilpy as yfu
import ysphotutilpy as ypu

import _Python_utilities
import _astro_utilities

plt.rcParams.update({'figure.max_open_warning': 0})

/tmp/ipykernel_167507/1580354273.py:19: DeprecationWarning: `photutils.DAOStarFinder` is a deprecated alias for `photutils.detection.DAOStarFinder` and will be removed in the future. Instead, please use `from photutils.detection import DAOStarFinder` to silence this warning.
  from photutils import DAOStarFinder
/tmp/ipykernel_167507/1580354273.py:20: DeprecationWarning: `photutils.IRAFStarFinder` is a deprecated alias for `photutils.detection.IRAFStarFinder` and will be removed in the future. Instead, please use `from photutils.detection import IRAFStarFinder` to silence this warning.
  from photutils import IRAFStarFinder
/tmp/ipykernel_167507/1580354273.py:23: DeprecationWarning: `photutils.detect_threshold` is a deprecated alias for `photutils.segmentation.detect_threshold` and will be removed in the future. Instead, please use `from photutils.segmentation import detect_threshold` to silence this warning.
  from photutils import detect_threshold


# 환경 설정

In [11]:
#%%
#######################################################
BASEDIR = Path("/mnt/Rdata/ASTRO_data")  

PROJECDIR = BASEDIR / "C1-Variable"
TODODIR = PROJECDIR / "-_-_-_2016-_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2017-03_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2017-06_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2021-10_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2022-01_-_RiLA600_STX-16803_-_2bin"

PROJECDIR = BASEDIR / "C2-Asteroid"
TODODIR = PROJECDIR / "-_-_-_2022-_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2023-_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_2bin"

# PROJECDIR = BASEDIR / "C3-EXO"
# TODODIR = PROJECDIR / "-_-_-_2024-05_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-05_-_RiLA600_STX-16803_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-09_-_RiLA600_ASI6200MMPro_-_2bin"

# PROJECDIR = BASEDIR / "C4-Spectra"
# TODODIR = PROJECDIR / "-_-_-_2024-05_TEC140_ASI183MMPro_-_1bin"

PROJECDIR = BASEDIR / "C5-Test"
TODODIR = PROJECDIR / "-_-_-_-_GSON300_STF-8300M_-_1bin"

DOINGDIRs = sorted(_Python_utilities.getFullnameListOfsubDirs(TODODIR))
print ("DOINGDIRs: ", format(DOINGDIRs))
print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

try : 
    BDFDIR = [x for x in DOINGDIRs if "CAL-BDF" in str(x)]
    print ("BDFDIR: ", format(BDFDIR))
    BDFDIR = Path(BDFDIR[0])    
except : 
    BDFDIR = TODODIR
    pass

DOINGDIRs = sorted([x for x in DOINGDIRs if "_LIGHT_" in str(x)])
# print ("DOINGDIRs: ", format(DOINGDIRs))
# print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

# filter_str = '127JOHANNA_LIGHT_-_2023-11-17_-_GSON300_STF-8300M_-_1bin'
# DOINGDIRs = [x for x in DOINGDIRs if filter_str in str(x)]
# remove = 'BIAS'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'DARK'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'FLAT'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
print ("DOINGDIRs: ", DOINGDIRs)
print ("len(DOINGDIRs): ", len(DOINGDIRs))
#######################################################

DOINGDIRs:  ['/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_STF-8300M_-_1bin/-_CAL-BDF_-_2024_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_STF-8300M_-_1bin/300GERALDINA_LIGHT_-_2023-12-22_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_STF-8300M_-_1bin/300GERALDINA_LIGHT_-_2024-01-12_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_STF-8300M_-_1bin/300GERALDINA_LIGHT_-_2024-01-13_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_STF-8300M_-_1bin/KELT-1b_LIGHT_-_2024-09-23_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_STF-8300M_-_1bin/KELT-1b_LIGHT_-_2024-11-23_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_STF-8300M_-_1bin/Kelt-1b_LIGHT_-_2024-10-04_-_GSON300_STF-8300M_-_1bin/']
len(DOINGDIRs):  7
BDFDIR:  ['/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_STF-8300M_-_1bin/-_CAL-BDF_-_2024_-_GSON300_STF-8300M_-_1bi

In [12]:
#%%
#####################################################################
# Observed location
LOCATION = dict(lon=127.005, lat=37.308889, elevation=101)
GSHS = EarthLocation(lon=127.005 * u.deg,
                                 lat=37.308889 * u.deg,
                                 height=101 * u.m)
MPC_obscode = "P64"
#######################################################
# Used for any `astropy.SkyCoord` object:
SKYC_KW = dict(unit=u.deg, frame='icrs')

# Initial guess of FWHM in pixel
FWHM_INIT = 4

FWHM = FWHM_INIT

# Photometry parameters
R_AP = 1.5 * FWHM_INIT # Aperture radius
R_IN = 4 * FWHM_INIT   # Inner radius of annulus
R_OUT = 6 * FWHM_INIT  # Outer radius of annulus
#######################################################

In [13]:
#%%
for DOINGDIR in DOINGDIRs[:1] :
    DOINGDIR = Path(DOINGDIR)
    print("DOINGDIR", DOINGDIR)
    DAOFINDERDIR = DOINGDIR / _astro_utilities.DAOfinder_result_dir
    if not DAOFINDERDIR.exists():
        os.makedirs("{}".format(str(DAOFINDERDIR)))
        print("{} is created...".format(str(DAOFINDERDIR)))
    
    summary = yfu.make_summary(DOINGDIR/"*.fit*",
                                    verify_fix=True,
                                    ignore_missing_simple=True,
                                    )
    if summary is not None :
        print("len(summary):", len(summary))
        print("summary:", summary)
        #print(summary["file"][0])  
        df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
        df_light = df_light.reset_index(drop=True)
        print("df_light:\n{}".format(df_light))
    df_light

DOINGDIR /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_STF-8300M_-_1bin/300GERALDINA_LIGHT_-_2023-12-22_-_GSON300_STF-8300M_-_1bin
/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_STF-8300M_-_1bin/300GERALDINA_LIGHT_-_2023-12-22_-_GSON300_STF-8300M_-_1bin/DAOfinder_result is created...
All 72 keywords (guessed from /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_STF-8300M_-_1bin/300GERALDINA_LIGHT_-_2023-12-22_-_GSON300_STF-8300M_-_1bin/300GERALDINA_LIGHT_B_2023-12-22-12-12-45_300sec_GSON300_STF-8300M_-29c_1bin.fit) will be loaded.


/home/guitar79/Downloads/ysfitsutilpy/ysfitsutilpy/filemgmt.py:347: UserWarning: Key CTYPE1 not found for /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_STF-8300M_-_1bin/300GERALDINA_LIGHT_-_2023-12-22_-_GSON300_STF-8300M_-_1bin/300GERALDINA_LIGHT_B_2023-12-22-13-45-59_300sec_GSON300_STF-8300M_-30c_1bin.fit, filling with None.
  warn(str_keyerror_fill.format(k, str(item)))
/home/guitar79/Downloads/ysfitsutilpy/ysfitsutilpy/filemgmt.py:347: UserWarning: Key CTYPE2 not found for /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_STF-8300M_-_1bin/300GERALDINA_LIGHT_-_2023-12-22_-_GSON300_STF-8300M_-_1bin/300GERALDINA_LIGHT_B_2023-12-22-13-45-59_300sec_GSON300_STF-8300M_-30c_1bin.fit, filling with None.
  warn(str_keyerror_fill.format(k, str(item)))
/home/guitar79/Downloads/ysfitsutilpy/ysfitsutilpy/filemgmt.py:347: UserWarning: Key CUNIT1 not found for /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_STF-8300M_-_1bin/300GERALDINA_LIGHT_-_2023-12-22_-_GSON300_STF-8300M_-_1bin/300GERALDINA_LIGHT_B_2

len(summary): 30
summary:                                                  file  filesize  SIMPLE  \
0   /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...  16983360    True   
1   /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...  16983360    True   
2   /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...  16983360    True   
3   /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...  16983360    True   
4   /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...  16983360    True   
5   /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...  16983360    True   
6   /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...  16980480    True   
7   /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...  16983360    True   
8   /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...  16983360    True   
9   /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...  16983360    True   
10  /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...  16983360    True   
11  /mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST

In [15]:
summary.columns

Index(['file', 'filesize', 'SIMPLE', 'BITPIX', 'NAXIS', 'NAXIS1', 'NAXIS2',
       'EXTEND', 'BZERO', 'IMAGETYP', 'EXPOSURE', 'EXPTIME', 'DATE-LOC',
       'DATE-OBS', 'DATE-AVG', 'XBINNING', 'YBINNING', 'OFFSET', 'EGAIN',
       'XPIXSZ', 'YPIXSZ', 'INSTRUME', 'SET-TEMP', 'CCD-TEMP', 'USBLIMIT',
       'TELESCOP', 'FOCALLEN', 'FOCRATIO', 'RA', 'DEC', 'CENTALT', 'CENTAZ',
       'AIRMASS', 'PIERSIDE', 'SITEELEV', 'SITELAT', 'SITELONG', 'FWHEEL',
       'FILTER', 'OBJECT', 'OBJCTRA', 'OBJCTDEC', 'OBJCTROT', 'FOCNAME',
       'FOCPOS', 'FOCUSPOS', 'FOCUSSZ', 'FOCTEMP', 'FOCUSTEM', 'ROWORDER',
       'EQUINOX', 'SWCREATE', 'CCDNAME', 'OPTIC', 'APATURE', 'PIXSCALE',
       'FLIPSTAT', 'CTYPE1', 'CTYPE2', 'CUNIT1', 'CRPIX1', 'CRPIX2', 'CRVAL1',
       'CRVAL2', 'CDELT1', 'CDELT2', 'CROTA1', 'CROTA2', 'CD1_1', 'CD1_2',
       'CD2_1', 'CD2_2', 'PLTSOLVD', 'WARNING'],
      dtype='object')

In [17]:
summary[['file', 'CD1_1', 'CD1_2', 'CD2_1', 'CD2_2']]

,file,CD1_1,CD1_2,CD2_1,CD2_2
0,/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...,-0.000254,-0.000007,0.000008,-0.000255
1,/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...,-0.000255,-0.000007,0.000007,-0.000255
2,/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...,-0.000255,-0.000007,0.000007,-0.000255
3,/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...,-0.000253,-0.000006,0.000007,-0.000255
4,/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...,-0.000253,-0.000006,0.000006,-0.000255
5,/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...,-0.000254,-0.000006,0.000007,-0.000255
6,/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...,NaN,NaN,NaN,NaN
7,/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...,0.000255,0.000009,-0.000009,0.000255
8,/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...,0.000255,0.000009,-0.000009,0.000254
9,/mnt/Rdata/ASTRO_data/C5-Test/-_-_-_-_GSON300_ST...,0.000255,0.000009,-0.000009,0.000255


In [ ]:
for _, row  in df_light.iterrows():
    fpath = Path(row["file"])
    # fpath = Path(df_light["file"][1])
    print("fpath :" ,fpath)
    hdul = fits.open(fpath)

    ## thres
    # thresh = detect_threshold(data=hdul[0].data, nsigma=3)
    # thresh = thresh[0][0]
    # print('detect_threshold', thresh)

    avg, med, std = sigma_clipped_stats(hdul[0].data)  # by default, 3-sigma 5-iteration.
    thresh = 5. * std
    # print('detect_threshold', thresh)

    DAOfind = DAOStarFinder(
                            fwhm = FWHM, 
                            threshold = thresh, 
                            # sharplo = 0.2, sharphi = 1.0,  # default values: sharplo=0.2, sharphi=1.0,
                            # roundlo = -1.0, roundhi = 1.0,  # default values -1 and +1
                            # sigma_radius = 1.5,           # default values 1.5
                            # ratio = 1.0,                  # 1.0: circular gaussian
                            exclude_border = True         # To exclude sources near edges
                            )
    # The DAOStarFinder object ("DAOfind") gets at least one input: the image.
    # Then it returns the astropy table which contains the aperture photometry results:
    DAOfound = DAOfind(hdul[0].data)

    print("DAOfound :", DAOfound)
    print("len(DAOfound) :",len(DAOfound))
    print(DAOfound.colnames)

    df_DAO = DAOfound.to_pandas()
    print(type(df_DAO))
    df_DAO.to_csv(f"{DAOFINDERDIR}/{fpath.stem}_DAOfinder_fwhm_{FWHM}.csv")
    print("df_DAO.describe :", df_DAO.describe)

    pos = np.transpose((DAOfound['xcentroid'], DAOfound['ycentroid']))
    apert = CAp(pos, r=R_AP)
    annul = CAn(positions=pos, r_in= R_IN, r_out=R_OUT)

    fig, axs = plt.subplots(1, 1, figsize=(12, 8),
                        # subplot_kw={'projection': wcs},
                        sharex=False, sharey=False, gridspec_kw=None)

    im = _astro_utilities.zimshow(axs, hdul[0].data, )

    axs.tick_params(labelsize=8)

    annul.plot(axs, color="r")
    for i in range(len(pos)):
        axs.text(pos[i][0], pos[i][1], f"Star #{str(i)}", fontsize=6, color='w')

    annul.plot(axs, color="r")

    cbar = plt.colorbar(im, ax = axs, fraction=0.035, pad=0.04, )
    cbar.ax.tick_params(labelsize=8)

    axs.set_title(f"fname: {fpath.name}\n Result of DAOFinder", fontsize=10,)

    axs.annotate(f'FWHM: {FWHM}', fontsize=8,
        xy=(0, 0), xytext=(-10, -30), va='top', ha='left',
        xycoords='axes fraction', textcoords='offset points')

    axs.annotate(f'Sky threshold: {thresh:.02f}', fontsize=8,
        xy=(0, 0), xytext=(-10, -40), va='top', ha='left',
        xycoords='axes fraction', textcoords='offset points')

    axs.annotate(f'Number of star(s): {len(DAOfound)}', fontsize=8,
        xy=(0, 0), xytext=(-10, -50), va='top', ha='left',
        xycoords='axes fraction', textcoords='offset points')

    plt.tight_layout()
    plt.savefig(f"{DAOFINDERDIR}/{fpath.stem}_DAOfinder_fwhm_{FWHM}.png")

    plt.show()


NameError: name 'df_light' is not defined

In [ ]:
df_DAO.describe

86